In [16]:
# 準備
import CaboCha

c = CaboCha.Parser()
with open("./neko.txt", "r") as rf:
    with open("./neko.txt.cabocha", "w") as wf:
        for line in rf:
            tree = c.parse(line.lstrip())
            wf.write(tree.toString(CaboCha.FORMAT_LATTICE))

In [21]:
# 40. 係り受け解析結果の読み込み（形態素）
class Morph(object):
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
        
    def __str__(self):
        return "surface: {}, base: {}, pos: {}, pos1: {}".format(self.surface, self.base, self.pos, self.pos1)

sentence_list = []
sentence = []
with open("./neko.txt.cabocha", "r") as f:
    for line in f:
        if line == "EOS\n":
            sentence_list.append(sentence)
            sentence = []
            continue
        elif line.split()[0] == "*":
            continue
        surface, features = line.split("\t")
        morph = Morph(surface=surface, base=features.split(",")[6], pos=features.split(",")[0], pos1=features.split(",")[1])
        sentence.append(morph)
        
for morph in sentence_list[2]:
    print(str(morph))
        
        

surface: 吾輩, base: 吾輩, pos: 名詞, pos1: 代名詞
surface: は, base: は, pos: 助詞, pos1: 係助詞
surface: 猫, base: 猫, pos: 名詞, pos1: 一般
surface: で, base: だ, pos: 助動詞, pos1: *
surface: ある, base: ある, pos: 助動詞, pos1: *
surface: 。, base: 。, pos: 記号, pos1: 句点


In [147]:
# 41. 係り受け解析結果の読み込み（文節・係り受け）
class Chunk(object):
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = int(dst.strip("D"))
        self.srcs = int(srcs)
        
    def get_chunk_text(self):
        text = ""
        for morph in self.morphs:
            if not morph.pos == "記号":
                text += morph.surface
        return text
    
    def has_noun(self):
        for morph in self.morphs:
            if morph.pos == "名詞":
                return True
        return False
    
    def has_verb(self):
        for morph in self.morphs:
            if morph.pos == "動詞":
                return True
        return False
    
    def has_particle(self):
        for morph in self.morphs:
            if morph.pos == "助詞":
                return True
        return False
    
    def is_sahen_wo(self):
        return len(self.morphs) == 2 and self.morphs[0].pos1 == "サ変接続" and self.morphs[1].base == "を"
    
    def get_last_particle(self):
        return [_morph for _morph in self.morphs if _morph.pos == '助詞'][-1] if self.has_particle() else None
    
    def get_first_verb(self):
        return [_morph for _morph in self.morphs if _morph.pos == '動詞'][0] if self.has_verb() else None
    
    
    def replace_noun(self, alt):
        for _morph in self.morphs:
            if _morph.pos == '名詞':
                _morph.surface = alt


sentence_list = []
sentence = []
chunk = None
with open("./neko.txt.cabocha", "r") as f:
    for line in f:
        line_list = line.split()
        if line_list[0] == "*":
            if chunk is not None:
                sentence.append(chunk)
            chunk = Chunk(morphs=[], dst=line_list[2], srcs=line_list[1])
        elif line_list[0] == "EOS":
            if chunk is not None:
                sentence.append(chunk)
            if len(sentence) > 0:
                sentence_list.append(sentence)
            sentence = []
            chunk = None
            continue
        else:
            surface, features = line.split("\t")
            morph = Morph(surface=surface, base=features.split(",")[6], pos=features.split(",")[0], pos1=features.split(",")[1])
            chunk.morphs.append(morph)
            
for chunk in sentence_list[7]:
    text = ""
    for morph in chunk.morphs:
        text += morph.surface
    print("文節番号 : " + str(chunk.srcs) + ", 文字列: " + text + ", 係り先文節番号 " + str(chunk.dst))

文節番号 : 0, 文字列: この, 係り先文節番号 1
文節番号 : 1, 文字列: 書生というのは, 係り先文節番号 7
文節番号 : 2, 文字列: 時々, 係り先文節番号 4
文節番号 : 3, 文字列: 我々を, 係り先文節番号 4
文節番号 : 4, 文字列: 捕えて, 係り先文節番号 5
文節番号 : 5, 文字列: 煮て, 係り先文節番号 6
文節番号 : 6, 文字列: 食うという, 係り先文節番号 7
文節番号 : 7, 文字列: 話である。, 係り先文節番号 -1


In [148]:
# 42. 係り元と係り先の文節の表示
for sentence in sentence_list[6:8]:
    print("=====文=====")
    for chunk in sentence:
        if not chunk.dst == -1:
            print(chunk.get_chunk_text() + "\t" + sentence[chunk.dst].get_chunk_text())

=====文=====
しかも	種族であったそうだ
あとで	聞くと
聞くと	種族であったそうだ
それは	種族であったそうだ
書生という	人間中で
人間中で	種族であったそうだ
一番	獰悪な
獰悪な	種族であったそうだ
=====文=====
この	書生というのは
書生というのは	話である
時々	捕えて
我々を	捕えて
捕えて	煮て
煮て	食うという
食うという	話である


In [149]:
# 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
for sentence in sentence_list[5:8]:
    print("=====文=====")
    for chunk in sentence:
        if not chunk.dst == -1 and chunk.has_noun() and sentence[chunk.dst].has_verb():
            print(chunk.get_chunk_text() + "\t" + sentence[chunk.dst].get_chunk_text())

=====文=====
吾輩は	見た
ここで	始めて
ものを	見た
=====文=====
あとで	聞くと
=====文=====
我々を	捕えて


In [152]:
# 44. 係り受け木の可視化
import pydot
for idx, sentence in enumerate(sentence_list[5:8]):
    nodes = ['"{}"->"{}"; '.format(chunk.get_chunk_text(), sentence[chunk.dst].get_chunk_text()) for chunk in sentence if  not chunk.dst == -1]
    sentence_dot = "digraph sentence{" + "".join(nodes) + "}"
    g = pydot.graph_from_dot_data(sentence_dot)
    g[0].write_jpeg("./jpg/sentence{}.jpg".format(idx))

In [153]:
# 45. 動詞の格パターンの抽出
# ("見る", ["は", "を"])というタプルのリスト
case_pattern_list = []
for sentence in sentence_list:
    for chunk in sentence:
        if not chunk.has_verb():
            continue
        particles = [ch.get_last_particle().base for ch in sentence if ch.dst == chunk.srcs and ch.has_particle()]
        if len(particles) == 0:
            continue
        particles.sort()
        case_pattern = (chunk.get_first_verb().base, particles)
        case_pattern_list.append(case_pattern)
        
with open("./case_patterns.txt", "w") as f:
    for case_pattern in case_pattern_list:
        f.write("{}\t{}\n".format(case_pattern[0], " ".join(case_pattern[1])))

In [154]:
# 46. 動詞の格フレーム情報の抽出
# ("見る", ["は", "を"], ["吾輩は", "ものを"])というタプルのリスト
case_frame_pattern_list = []
for sentence in sentence_list[5:8]:
    for chunk in sentence:
        if not chunk.has_verb():
            continue
        particles = [(ch.get_last_particle().base, ch.get_chunk_text()) for ch in sentence if ch.dst == chunk.srcs and ch.has_particle()]
        if len(particles) == 0:
            continue
        sorted_particles = sorted(particles, key=lambda x:x[0])
        particle_list, chunk_text_list = [], []
        for t in sorted_particles:
            particle_list.append(t[0])
            chunk_text_list.append(t[1])
        case_pattern = (chunk.get_first_verb().base, particle_list, chunk_text_list)
        case_frame_pattern_list.append(case_pattern)
        
for cfp in case_frame_pattern_list:
    print(cfp[0] + "\t" + " ".join(cfp[1]) + "\t" + " ".join(cfp[2]))

始める	で	ここで
見る	は を	吾輩は ものを
聞く	で	あとで
捕える	を	我々を
煮る	て	捕えて
食う	て	煮て


In [155]:
# 47. 機能動詞構文のマイニング
sahen_frame_pattern_list = []
for sentence in sentence_list[800:1000]:
    for chunk in sentence:
        if not chunk.has_verb():
            continue
        particles = [(ch.get_last_particle().base, ch) for ch in sentence if ch.dst == chunk.srcs and ch.has_particle()]
        if len(particles) == 1:
            continue
        # 動詞に「サ変接続+を」が係るかチェック
        has_sahen_wo = False
        for particle in particles:
            if  particle[1].is_sahen_wo():
                has_sahen_wo = True
                sahen_wo = particle[1].get_chunk_text()
        if not has_sahen_wo:
            continue
        sorted_particles = sorted(particles, key=lambda x:x[0])
        particle_list, chunk_text_list = [], []
        for t in sorted_particles:
            if t[1].get_chunk_text() == sahen_wo:
                continue
            particle_list.append(t[0])
            chunk_text_list.append(t[1].get_chunk_text())
        sahen_pattern = (sahen_wo + chunk.get_first_verb().base, particle_list, chunk_text_list)
        sahen_frame_pattern_list.append(sahen_pattern)
        
for cfp in sahen_frame_pattern_list:
    print(cfp[0] + "\t" + " ".join(cfp[1]) + "\t" + " ".join(cfp[2]))

研究を続ける	て て	組織しまして 会合して
同情を持つ	に	人物に
質問をする	が て	したが 見えて
御馳走を致す	に	東風子に
返事をする	と に は	及ばんさと 手紙に 主人は
著述を居る	て より	依て 此間中より
厚遇を受ける	が まで	猫が かくまで
返事をする	は	下女は
含嗽をやる	で	毎朝風呂場で


In [156]:
# 48. 名詞から根へのパスの抽出
def get_path_list(sentence, idx, paths):
    if sentence[idx].dst != -1:
        paths.append(sentence[idx])
        get_path_list(sentence, sentence[idx].dst, paths)
    else:
        paths.append(sentence[idx])
    return paths


paths_list = []
for sentence in sentence_list[5:8]:
    for chunk in sentence:
        if chunk.has_noun():
            paths = get_path_list(sentence, chunk.srcs, [])
            if len(paths) == 1:
                continue
            paths_list.append(paths)
            
for paths in paths_list:
    print(" -> ".join([ch.get_chunk_text() for ch in paths]))

吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た
あとで -> 聞くと -> 種族であったそうだ
それは -> 種族であったそうだ
書生という -> 人間中で -> 種族であったそうだ
人間中で -> 種族であったそうだ
一番 -> 獰悪な -> 種族であったそうだ
獰悪な -> 種族であったそうだ
書生というのは -> 話である
我々を -> 捕えて -> 煮て -> 食うという -> 話である


In [160]:
# 49. 名詞間の係り受けパスの抽出
from itertools import combinations

def noun_pairs(sen):
    noun_chunk_list = [ch for ch in sen if ch.has_noun()]
    return list(combinations(noun_chunk_list, 2))


def common_chunk(path_i, path_j):
    chunk_k = None
    path_i = list(reversed(path_i))
    path_j = list(reversed(path_j))
    for idx, (c_i, c_j) in enumerate(zip(path_i, path_j)):
        if c_i.srcs != c_j.srcs:
            chunk_k = path_i[idx - 1]
            break

    return chunk_k


for sentence in sentence_list[5:8]:
    n_pairs = noun_pairs(sentence)
    if len(n_pairs) == 0:
        continue
    
    for n_pair in n_pairs:
        chunk_i, chunk_j = n_pair
        chunk_i.replace_noun('X')
        chunk_j.replace_noun('Y')
        
        paths_i = get_path_list(sentence, chunk_i.srcs, [])
        paths_j = get_path_list(sentence, chunk_j.srcs, [])
        
        if chunk_j in paths_i:
            # 文節iiから構文木の根に至る経路上に文節jjが存在する場合
            idx_j = paths_i.index(chunk_j)
            print(" -> ".join([ch.get_chunk_text() for ch in paths_i[:idx_j + 1]]))
            
        else:
            # 文節iiと文節jjから構文木の根に至る経路上で共通の文節kkで交わる場合
            chunk_k = common_chunk(paths_i, paths_j)
            if chunk_k is None:
                continue
            idx_k_i = paths_i.index(chunk_k)
            idx_k_j = paths_j.index(chunk_k)
            print(" | ".join([" -> ".join([ch.get_chunk_text() for ch in paths_i[:idx_k_i + 1]]),
                             " -> ".join([ch.get_chunk_text() for ch in paths_j[:idx_k_j + 1]]),
                             chunk_k.get_chunk_text()]))

Xは -> 見た | Yで -> 始めて -> Yという -> Yを -> 見た | 見た
Xは -> 見た | Yという -> Yを -> 見た | 見た
Xは -> 見た | Yを -> 見た | 見た
Xで -> 始めて -> Yという
Xで -> 始めて -> Yという -> Yを
Xという -> Yを
Xで -> 聞くと -> 種族であったそうだ | Yは -> 種族であったそうだ | 種族であったそうだ
Xで -> 聞くと -> 種族であったそうだ | Yという -> 人間中で -> 種族であったそうだ | 種族であったそうだ
Xで -> 聞くと -> 種族であったそうだ | YYで -> 種族であったそうだ | 種族であったそうだ
Xで -> 聞くと -> 種族であったそうだ | Y -> 獰悪な -> 種族であったそうだ | 種族であったそうだ
Xで -> 聞くと -> 種族であったそうだ | Yな -> 種族であったそうだ | 種族であったそうだ
Xで -> 聞くと -> YであったYだ
Xは -> YであったYだ | Yという -> YYで -> YであったYだ | YであったYだ
Xは -> YであったYだ | YYで -> YであったYだ | YであったYだ
Xは -> YであったYだ | Y -> Yな -> YであったYだ | YであったYだ
Xは -> YであったYだ | Yな -> YであったYだ | YであったYだ
Xは -> YであったYだ
Xという -> YYで
Xという -> YYで -> YであったYだ | Y -> Yな -> YであったYだ | YであったYだ
Xという -> YYで -> YであったYだ | Yな -> YであったYだ | YであったYだ
Xという -> YYで -> YであったYだ
XXで -> YであったYだ | Y -> Yな -> YであったYだ | YであったYだ
XXで -> YであったYだ | Yな -> YであったYだ | YであったYだ
XXで -> YであったYだ
X -> Yな
X -> Yな -> YであったYだ
Xな -> YであったYだ
XというXは -> 話である | Yを -> 捕えて -> 煮て -> 食うという -> 話である | 話である
XというXは -> Yであ